# Finding missions of interest

In [1]:
import pandas as pd
import numpy as np
from erddapy import ERDDAP

For a quick search, We can use ERDDAP's built in search function to find datasets that match a keyword

In [2]:
e = ERDDAP(
    server="https://erddap.observations.voiceoftheocean.org/erddap"
)
url = e.get_search_url(search_for="cable", response="csv")

df = pd.read_csv(url)
df["Dataset ID"]

0         nrt_SEA067_M26
1         nrt_SEA067_M27
2         nrt_SEA067_M29
3         nrt_SEA066_M45
4         nrt_SEA067_M30
5         nrt_SEA067_M32
6         nrt_SEA066_M41
7         nrt_SEA066_M42
8         nrt_SEA066_M43
9          nrt_SEA079_M9
10        nrt_SEA076_M13
11        nrt_SEA078_M11
12        nrt_SEA067_M37
13        nrt_SEA067_M39
14    delayed_SEA067_M26
15    delayed_SEA067_M27
16    delayed_SEA067_M29
17    delayed_SEA067_M30
18    delayed_SEA067_M32
19    delayed_SEA066_M43
20    delayed_SEA066_M41
21    delayed_SEA066_M42
22    delayed_SEA076_M13
23     delayed_SEA079_M9
24    delayed_SEA078_M11
25    delayed_SEA067_M37
26    delayed_SEA067_M39
27       adcp_SEA067_M26
28       adcp_SEA067_M27
29       adcp_SEA067_M29
30        adcp_SEA079_M9
31       adcp_SEA076_M13
32       adcp_SEA067_M30
33       adcp_SEA067_M32
34       adcp_SEA067_M37
35       adcp_SEA067_M39
36       adcp_SEA066_M41
37       adcp_SEA066_M42
38       adcp_SEA066_M43
39       adcp_SEA078_M11


### Load metadata table

For more advanced searches, we load the dedicated datasets "meta_users_table" to get metadata on all datasets hosted on the VOTO ERDDAP

**NB** this notebook uses a dedicated data table not available on most ERDDAP servers. For a more general treatment of gathering metadata on all datasets on a server, see the notebook `legacy_01_mission_filter.ipynb`

In [3]:
e = ERDDAP(server="https://erddap.observations.voiceoftheocean.org/erddap", protocol='tabledap')

In [4]:
e.response = "csv"
e.dataset_id = "meta_users_table"
table = e.to_pandas()

In [5]:
table

,glider_serial,deployment_id,basin,deployment_start (UTC),deployment_end (UTC),available_variables,science_variables,ctd,oxygen,optics,ad2cp,irradiance,nitrate,datasetID
0,SEA068,27,Eastern Gotland Basin,2022-07-27T00:00:00Z,2022-07-31T00:00:00Z,"['profile_index', 'rowSize', 'latitude', 'long...","['altimeter', 'backscatter_raw', 'backscatter_...","{'calibration_date': '2021-06-17', 'factory_ca...","{'calibration_date': '2021-06-17', 'factory_ca...","{'calibration_date': '2022-05-10', 'calibratio...",NaN,NaN,NaN,nrt_SEA068_M27
1,SEA076,8,Bornholm Basin,2022-10-05T00:00:00Z,2022-10-19T00:00:00Z,"['profile_index', 'rowSize', 'latitude', 'long...","['ad2cp_beam1_cell_number1', 'ad2cp_beam2_cell...","{'calibration_date': '2022-05-16', 'factory_ca...","{'calibration_date': '2022-05-26', 'factory_ca...","{'calibration_date': '2022-04-04', 'calibratio...","{'calibration_date': '2022-03-16', 'factory_ca...",NaN,"{'calibration_date': '2019-05-20', 'factory_ca...",nrt_SEA076_M8
2,SEA076,9,Bornholm Basin,2022-10-20T00:00:00Z,2022-10-31T00:00:00Z,"['profile_index', 'rowSize', 'latitude', 'long...","['ad2cp_beam1_cell_number1', 'ad2cp_beam2_cell...","{'calibration_date': '2022-05-16', 'factory_ca...","{'calibration_date': '2022-05-26', 'factory_ca...","{'calibration_date': '2022-04-04', 'calibratio...","{'calibration_date': '2022-03-16', 'factory_ca...",NaN,"{'calibration_date': '2019-05-20', 'factory_ca...",nrt_SEA076_M9
3,SEA076,13,"Northern Baltic Proper, Eastern Gotland Basin",2023-01-13T00:00:00Z,2023-02-14T00:00:00Z,"['profile_index', 'rowSize', 'latitude', 'long...","['ad2cp_beam1_cell_number1', 'ad2cp_beam2_cell...","{'calibration_date': '2021-03-01', 'factory_ca...","{'calibration_date': '2020-02-05', 'factory_ca...","{'calibration_date': '2022-04-04', 'calibratio...","{'calibration_date': '2022-03-16', 'factory_ca...",NaN,NaN,nrt_SEA076_M13
4,SEA076,16,"Eastern Gotland Basin, Northern Baltic Proper",2023-04-11T00:00:00Z,2023-05-16T00:00:00Z,"['profile_index', 'rowSize', 'latitude', 'long...","['ad2cp_beam1_cell_number1', 'ad2cp_beam2_cell...","{'calibration_date': '2021-03-01', 'factory_ca...","{'calibration_date': '2022-11-08', 'factory_ca...","{'calibration_date': '2022-04-04', 'calibratio...","{'calibration_date': '2022-03-16', 'factory_ca...",NaN,NaN,nrt_SEA076_M16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,SEA078,11,"Northern Baltic Proper, Eastern Gotland Basin",2023-03-16T00:00:00Z,2023-04-11T00:00:00Z,"['profile_index', 'rowSize', 'latitude', 'long...","['ad2cp_beam1_cell_number1', 'ad2cp_beam2_cell...","{'calibration_date': '2022-10-24', 'factory_ca...","{'calibration_date': '2022-11-11', 'factory_ca...","{'calibration_date': '2022-10-05', 'calibratio...","{'calibration_date': '2022-08-02', 'factory_ca...",NaN,NaN,nrt_SEA078_M11
178,SEA078,12,Eastern Gotland Basin,2023-05-16T00:00:00Z,2023-06-20T00:00:00Z,"['profile_index', 'rowSize', 'latitude', 'long...","['ad2cp_beam1_cell_number1', 'ad2cp_beam2_cell...","{'calibration_date': '2022-10-24', 'factory_ca...","{'calibration_date': '2022-11-11', 'factory_ca...","{'calibration_date': '2022-10-05', 'calibratio...","{'calibration_date': '2022-08-02', 'factory_ca...",NaN,NaN,nrt_SEA078_M12
179,SEA078,14,Western Gotland Basin,2023-07-28T00:00:00Z,2023-09-06T00:00:00Z,"['profile_index', 'rowSize', 'latitude', 'long...","['ad2cp_beam1_cell_number1', 'ad2cp_beam2_cell...","{'calibration_date': '2022-10-24', 'factory_ca...","{'calibration_date': '2022-11-08', 'factory_ca...","{'calibration_date': '2022-10-05', 'calibratio...","{'calibration_date': '2022-08-02', 'factory_ca...",NaN,NaN,nrt_SEA078_M14
180,SEA078,15,Western Gotland Basin,2023-10-06T00:00:00Z,2023-11-11T00:00:00Z,"['profile_index', 'rowSize', 'latitude', 'long...","['ad2cp_beam1_cell_number1', 'ad2cp_beam2_cell...","{'calibration_date': '2022-10-24', 'factory_ca...","{'calibration_date': '2022-11-08', 'factory_ca...","{'calibration_date': '2022-10-05', 'calibratio...","{'calibration_date': '2022-08

### List of all possible science variables

In [6]:
sci_variables_all = []

for i, row in table.iterrows():
    sci_variables_all.extend(eval(row.science_variables[:]))

res = []
[res.append(x) for x in sci_variables_all if x not in res]

nav_var = {'profile_index', 'rowSize', 'latitude', 'longitude', 'time', 'depth',
           'angular_cmd', 'angular_pos', 'ballast_cmd', 'ballast_pos', 'desired_heading',
           'dive_num', 'heading', 'internal_pressure', 'internal_temperature', 'linear_cmd',
           'linear_pos', 'nav_state', 'pitch', 'profile_direction', 'profile_num',
           'roll', 'security_level', 'vertical_distance_to_seafloor', 'voltage', 'declination'}

res_sci = [i for i in res if i not in nav_var]

res_sci

['altimeter',
 'backscatter_raw',
 'backscatter_scaled',
 'cdom',
 'cdom_qc',
 'cdom_raw',
 'cdom_raw_qc',
 'chlorophyll',
 'chlorophyll_qc',
 'chlorophyll_raw',
 'conductivity',
 'conductivity_qc',
 'dead_reckoning',
 'density',
 'density_qc',
 'distance_over_ground',
 'nav_resource',
 'oxygen_concentration',
 'oxygen_concentration_qc',
 'oxygen_phase',
 'particulate_backscatter',
 'potential_density',
 'potential_density_qc',
 'potential_temperature',
 'potential_temperature_qc',
 'pressure',
 'salinity',
 'salinity_qc',
 'temperature',
 'temperature_oxygen',
 'temperature_qc',
 'ad2cp_beam1_cell_number1',
 'ad2cp_beam2_cell_number1',
 'ad2cp_beam3_cell_number1',
 'ad2cp_beam4_cell_number1',
 'ad2cp_heading',
 'ad2cp_pitch',
 'ad2cp_pressure',
 'ad2cp_roll',
 'ad2cp_time',
 'nitrate_concentration',
 'nitrate_molar_concentration',
 'suna_internal_humidity',
 'suna_internal_temperature',
 'nanoflu_temperature',
 'phycocyanin',
 'phycocyanin_nanoflu',
 'phycocyanin_raw',
 'backscatter_r

## Examples of query

### Find all the mission where nitrate data was available

In [7]:
name = []
for i in range(len(table)):
    if 'nitrate_concentration' in table.science_variables[i]:
        name.append(f'{table.glider_serial[i]}_{table.deployment_id[i]}')
print(f"found {len(name)} datasets: {name}")

found 2 datasets: ['SEA076_8', 'SEA076_9']


### Find all the mission with the FLBBPC optical sensor

In [8]:
name = []
for i in range(len(table)):
    if 'FLBBPC' in eval(table.optics[i])['model']:
        name.append(f'{table.glider_serial[i]}_{table.deployment_id[i]}')
print(f"found {len(name)} datasets: {name}")

found 92 datasets: ['SEA069_9', 'SEA069_11', 'SEA069_13', 'SEA069_15', 'SEA069_17', 'SEA069_19', 'SEA069_21', 'SEA069_22', 'SEA069_23', 'SEA055_18', 'SEA055_19', 'SEA055_20', 'SEA055_21', 'SEA055_23', 'SEA055_24', 'SEA055_26', 'SEA055_28', 'SEA055_31', 'SEA055_33', 'SEA055_35', 'SEA055_37', 'SEA055_39', 'SEA055_41', 'SEA055_43', 'SEA055_45', 'SEA055_46', 'SEA055_48', 'SEA055_50', 'SEA055_52', 'SEA055_53', 'SEA055_54', 'SEA055_56', 'SEA045_79', 'SEA063_55', 'SEA063_60', 'SEA044_28', 'SEA044_29', 'SEA044_32', 'SEA044_33', 'SEA044_34', 'SEA044_35', 'SEA044_40', 'SEA044_45', 'SEA044_46', 'SEA044_48', 'SEA044_72', 'SEA044_75', 'SEA044_77', 'SEA044_78', 'SEA044_79', 'SEA044_81', 'SEA044_83', 'SEA044_84', 'SEA044_85', 'SEA044_86', 'SEA044_88', 'SEA067_15', 'SEA067_26', 'SEA067_27', 'SEA067_29', 'SEA067_30', 'SEA067_32', 'SEA067_37', 'SEA067_39', 'SEA067_41', 'SEA067_42', 'SEA067_44', 'SEA067_46', 'SEA067_51', 'SEA067_53', 'SEA066_10', 'SEA066_12', 'SEA066_14', 'SEA066_16', 'SEA066_41', 'SEA06

### Find all Eastern Gotland mission in the period Jan 2022 - March 2023

In [9]:
name = []
for i in range(len(table)):
    if 'Eastern Gotland Basin' in table.basin[i]:
        if table['deployment_start (UTC)'][i] > '2022-01-01' and table['deployment_start (UTC)'][i] < '2023-03-01':
            name.append(f'{table.glider_serial[i]}_{table.deployment_id[i]}')
print(f"found {len(name)} datasets: {name}")

found 14 datasets: ['SEA068_27', 'SEA076_13', 'SEA069_9', 'SEA079_9', 'SEA067_26', 'SEA067_27', 'SEA067_29', 'SEA067_30', 'SEA067_32', 'SEA067_37', 'SEA067_39', 'SEA066_41', 'SEA066_42', 'SEA066_43']


### Find all the mission with the oxygen sensor serial number 66

In [10]:
name = []
for i in range(len(table)):
    if '66' in eval(table.oxygen[i])['serial']:
        name.append(f'{table.glider_serial[i]}_{table.deployment_id[i]}')
print(f"found {len(name)} datasets: {name}")

found 8 datasets: ['SEA076_16', 'SEA076_17', 'SEA069_21', 'SEA069_22', 'SEA069_23', 'SEA077_21', 'SEA078_14', 'SEA078_15']


### Find which brand this oxygen sensor is and what the calibration parameters are and where the mission was

In [11]:
oxy_tab = table.copy()
for i in range(len(oxy_tab)):
    if '66' not in eval(oxy_tab.oxygen[i])['serial']:
        oxy_tab.iloc[i] = np.nan
oxy_tab.dropna(how='all')

,glider_serial,deployment_id,basin,deployment_start (UTC),deployment_end (UTC),available_variables,science_variables,ctd,oxygen,optics,ad2cp,irradiance,nitrate,datasetID
4,SEA076,16.0,"Eastern Gotland Basin, Northern Baltic Proper",2023-04-11T00:00:00Z,2023-05-16T00:00:00Z,"['profile_index', 'rowSize', 'latitude', 'long...","['ad2cp_beam1_cell_number1', 'ad2cp_beam2_cell...","{'calibration_date': '2021-03-01', 'factory_ca...","{'calibration_date': '2022-11-08', 'factory_ca...","{'calibration_date': '2022-04-04', 'calibratio...","{'calibration_date': '2022-03-16', 'factory_ca...",NaN,NaN,nrt_SEA076_M16
5,SEA076,17.0,Western Gotland Basin,2023-06-20T00:00:00Z,2023-07-28T00:00:00Z,"['profile_index', 'rowSize', 'latitude', 'long...","['ad2cp_beam1_cell_number1', 'ad2cp_beam2_cell...","{'calibration_date': '2021-03-01', 'factory_ca...","{'calibration_date': '2022-11-08', 'factory_ca...","{'calibration_date': '2022-04-04', 'calibratio...","{'calibration_date': '2022-03-16', 'factory_ca...",NaN,NaN,nrt_SEA076_M17
15,SEA069,21.0,"Skagerrak, Kattegat",2023-11-17T00:00:00Z,2023-12-01T00:00:00Z,"['profile_index', 'rowSize', 'latitude', 'long...","['altimeter', 'backscatter_raw', 'backscatter_...","{'calibration_date': '2023-06-12', 'factory_ca...","{'calibration_date': '2022-11-08', 'factory_ca...","{'calibration_date': '2021-03-22', 'calibratio...",NaN,NaN,NaN,nrt_SEA069_M21
16,SEA069,22.0,"Kattegat, Skagerrak",2023-12-02T00:00:00Z,2023-12-14T00:00:00Z,"['profile_index', 'rowSize', 'latitude', 'long...","['altimeter', 'backscatter_raw', 'backscatter_...","{'calibration_date': '2023-06-12', 'factory_ca...","{'calibration_date': '2022-11-08', 'factory_ca...","{'calibration_date': '2021-03-22', 'calibratio...",NaN,NaN,NaN,nrt_SEA069_M22
17,SEA069,23.0,"Skagerrak, Kattegat",2023-12-27T00:00:00Z,2024-01-18T00:00:00Z,"['profile_index', 'rowSize', 'latitude', 'long...","['altimeter', 'backscatter_raw', 'backscatter_...","{'calibration_date': '2023-06-12', 'factory_ca...","{'calibration_date': '2022-11-08', 'factory_ca...","{'calibration_date': '2021-03-22', 'calibratio...",NaN,NaN,NaN,nrt_SEA069_M23
147,SEA077,21.0,Eastern Gotland Basin,2023-03-16T00:00:00Z,2023-04-11T00:00:00Z,"['profile_index', 'rowSize', 'latitude', 'long...","['ad2cp_beam1_cell_number1', 'ad2cp_beam2_cell...","{'calibration_date': '2022-05-17', 'factory_ca...","{'calibration_date': '2022-11-08', 'factory_ca...","{'calibration_date': '2022-04-20', 'calibratio...","{'calibration_date': '2022-06-02', 'factory_ca...",NaN,NaN,nrt_SEA077_M21
179,SEA078,14.0,Western Gotland Basin,2023-07-28T00:00:00Z,2023-09-06T00:00:00Z,"['profile_index', 'rowSize', 'latitude', 'long...","['ad2cp_beam1_cell_number1', 'ad2cp_beam2_cell...","{'calibration_date': '2022-10-24', 'factory_ca...","{'calibration_date': '2022-11-08', 'factory_ca...","{'calibration_date': '2022-10-05', 'calibratio...","{'calibration_date': '2022-08-02', 'factory_ca...",NaN,NaN,nrt_SEA078_M14
180,SEA078,15.0,Western Gotland Basin,2023-10-06T00:00:00Z,2023-11-11T00:00:00Z,"['profile_index', 'rowSize', 'latitude', 'long...","['ad2cp_beam1_cell_number1', 'ad2cp_beam2_cell...","{'calibration_date': '2022-10-24', 'factory_ca...","{'calibration_date': '2022-11-08', 'factory_ca...","{'calibration_date': '2022-10-05', 'calibratio...","{'calibration_date': '2022-08-02', 'factory_ca...",NaN,NaN,nrt_SEA078_M15


### Find all types of optical sensors data available

In [12]:
opt_sens = []
for i, row in table.iterrows():
    opt_sens.append([eval(row.optics)['model'], eval(row.optics)['make']])

opt_sens_type = []
[opt_sens_type.append(x) for x in opt_sens if x not in opt_sens_type]

opt_sens_type

[['FLBBCD', 'Wetlabs'],
 ['FLBBPC', 'Wetlabs'],
 ['FLNTU', 'Wetlabs'],
 ['SEAOWL', 'Seabird']]

In [13]:
#List all FLBBPC sensors and calibration data
FLBBPC_sens = []
for i, row in table.iterrows():
    if 'FLBBPC' in eval(row.optics)['model']:
        FLBBPC_sens.append([eval(row.optics)['serial'], eval(row.optics)['calibration_date']])

FLBBPC_sens_type = []
[FLBBPC_sens_type.append(x) for x in FLBBPC_sens if x not in FLBBPC_sens_type]

FLBBPC_sens_type

[['6756', '2021-03-23'],
 ['6754', '2021-03-22'],
 ['6598', '2020-12-22'],
 ['6754', '2021-03-23'],
 ['6597', '2020-12-22'],
 ['6755', '2021-03-22'],
 ['7313', '2022-01-11'],
 ['6753', '2021-03-23']]

-------------------------------

**References**

The metadata table at the core of this notebook is regenerated every day using a script developed by @MOchiara https://github.com/voto-ocean-knowledge/data_updates